In [37]:
!pip install git+https://github.com/goolig/dsClass.git

# BGU Data Science Course

In [38]:
# imports:
import pandas as pd
from dsClass.path_helper import *

### load the data:

In [39]:
theft_data_path = get_file_path('theft_data_bgu_shuffled.csv')
theft_data = pd.read_csv(theft_data_path)
holiday_data_path = get_file_path('holidays_data_bgu.csv')
holidays_data_bgu = pd.read_csv(holiday_data_path)

In [40]:
# check the columns:
theft_data.columns

In [41]:
#Q1:

# Consider only important columns:
# Keep only the following columns in your dataframe- Date, District, Count(our label), Year

#### insert your code here:
theft_data=theft_data[['Date','District','Count','Year']]






############
#This is for us, in the end print the columns:
theft_data.columns

In [42]:
theft_data.head()

### Feature extraction:

In [43]:
#Q2:

# Change the Date from string to timestamp, in theft_data & holidays_data_bgu:
# Hint - use dt.datetime.strptime
import datetime as dt
#### insert your code here:



In [84]:
holidays_data_bgu

In [45]:
theft_data['Date'] = theft_data['Date'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%Y"))


In [46]:

holidays_data_bgu['Date'] = holidays_data_bgu['Date'].apply(lambda x:dt.datetime.strptime(x,"%Y-%m-%d"))


In [47]:
# add time information:

#Q3:

# Create day of week column:
# Name the column Week_day, it will contain a number for each day of week: 
# Monday=0, ...., Sunday=6
#### insert your code here:

theft_data_dayOfWeek=[x.weekday() for x in theft_data['Date']]
theft_data['Week_day']=theft_data_dayOfWeek
theft_data.head()

In [48]:
holidays_data_dayOfWeek=[x.weekday() for x in holidays_data_bgu['Date']]
holidays_data_bgu['Week_day']=holidays_data_dayOfWeek
holidays_data_bgu.head()

In [49]:
#Q4:
# Create season column:
# Name the column Season, it will contain a string which tells us the season based on the Date: 
# Hint - use the following package to indicate specific dates:
from datetime import date
# and write function that gets the date and returns the corresponding season
#### insert your code here:


# defining get_season function
Y = 2000
seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    now = now.replace(year=Y).date()
    
    return next(season for season, (start, end) in seasons
                if start <= now <= end)


[(get_season(holidays_data_bgu['Date'][i]), holidays_data_bgu['Date'][i]) for i in range(10)]

In [50]:
# add season column to theft_data
theft_data_Season=[get_season(x) for x in theft_data['Date']]
theft_data['Season']=theft_data_Season
theft_data.head()

In [51]:
# add season column to holidays_data_bgu
holidays_data_Season=[get_season(x) for x in holidays_data_bgu['Date']]
holidays_data_bgu['Season']=holidays_data_Season
holidays_data_bgu.head()

In [82]:
#Q5:

# Add information from other sources
# Holidays data
# Create column is_holiday that will take 1 if its an holiday and 0 otherwise.

#### insert your code here:

a = theft_data
b = holidays_data_bgu
b['is_holiday'] = [1 for x in holidays_data_bgu['Date']]
b = b[['Date', 'is_holiday']]

holidays_df = a.merge(b, on='Date', how='left')
holidays_df['is_holiday'] =holidays_df['is_holiday'].apply(lambda x: 1 if x == 1 else 0)

holidays_df.groupby('Date').sum('is_holiday')
theft_data =  holidays_df

# test results
# k = 0 
# for x in theft_data['Date']:
#     if x in b['Date'].unique():
#         k+= 1
# k


In [53]:
# Now you are supposed to have the following columns: Date, is_holiday, district, count(our label), year, week_day, season
theft_data.columns

In [61]:
#Q6:
# Create dummy variables:
# We create dummy variables because our model used later (linear regression) does not support categorical features
# Hint - use the pandas function 'get_dummies'
#### insert your code here:
theft_data = pd.get_dummies(theft_data, columns = ['Season'])
theft_data

In [62]:
# Now you suppose to have the following columns: Date, is_holiday, count, year,
# dummies for season, week_day and district


theft_data.columns

In [64]:
#Q7:

# remove the original date columns: (we won't use it for modelling)
#### insert your code here:
theft_data = theft_data.drop('Date',axis=1)
theft_data


### Train-Test Splitting:

In [57]:
# Choose years for train and test
train_start_year = 2014
train_end_year = 2015
test_year = 2016

In [76]:
# split the data into train/test
dataTrain = theft_data[(theft_data["Year"] >= train_start_year) & (theft_data["Year"] <= train_end_year)]
labelsTrain = dataTrain.Count
dataTrain = dataTrain.drop('Count', axis=1)

dataTest = theft_data[(theft_data["Year"] == test_year)]
labelsTest = dataTest.Count
dataTest = dataTest.drop('Count', axis=1)

# Remove unnecessary columns:
dataTrain = dataTrain.drop('Year', axis=1)
dataTest = dataTest.drop('Year', axis=1)

print("Train data shape: " , dataTrain.shape)
print("Test data shape: " , dataTest.shape)
print("Train data columns: ", dataTrain.columns)

In [78]:
# check for null values (should print 0)
print(theft_data.isnull().sum().sum())

### Modelling

In [77]:
from sklearn.linear_model import LinearRegression

mlModel = LinearRegression()

mlModel.fit(dataTrain, labelsTrain)

### Evaluation

In [79]:
predTest = mlModel.predict(dataTest)

# print the Rsquare:
print("Test set R^2: ", mlModel.score(dataTest, labelsTest))

### Question:

# Q8: 

Use [cross_validate](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) to evaluate the model. Make sure you read the documentation carefully and use the right parameters and data format for a strong evaluation to our data and model.



In [ ]:
#Q8:



# Q9:  
 Does this method perfectly fit our needs? 
 If yes explain why did you choose this method.
 If your answer is no explain why did you choose it and what it is the problem.